In [56]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import cv2

In [57]:
device = torch.device('cpu')

In [58]:
VGG_11 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)
class VGG_CAM(nn.Module):
    def __init__(self, features = VGG_11.features, n_classes = 3):
        super(VGG_CAM, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512, n_classes,bias=False)
            
    def forward(self, x):
        x = self.features(x)
        self.featuremap1 = x.detach()
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
preprocess = transforms.Compose([
    transforms.CenterCrop(1024),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

Using cache found in /home/cz2064/.cache/torch/hub/pytorch_vision_v0.6.0


In [59]:
model = VGG_CAM().to(device)
files = torch.load('./Python File/checkpoint_Model.pt', map_location='cpu')
model.load_state_dict(files['best_model_wts'])

/scratch/cz2064/envs/dl4med/lib/python3.6/site-packages/torch/serialization.py:559: UserWarning: Couldn't retrieve source code for container of type VGG_CAM. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
/scratch/cz2064/envs/dl4med/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/scratch/cz2064/envs/dl4med/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the 

<All keys matched successfully>

In [60]:
test_df_path = '/scratch/cz2064/myjupyter/BDML/Project/Phase5/Train_Test_Split/test.csv'
test_df = pd.read_csv(test_df_path)
test_df[6:]

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
6,bbbabc12-da9c-446a-8a5f-ef6a8db7cd0a,TCGA-64-5778-01A-01-TS1.e85c2800-9cf3-4e65-a00...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-64-5778,TCGA-64-5778-01A,Primary Tumor,Tumor-LUAD,2
7,da66ca5f-50c4-42b3-854a-31d006ae6341,TCGA-05-4389-01A-01-TS1.8609dd2e-0f65-4941-83f...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-05-4389,TCGA-05-4389-01A,Primary Tumor,Tumor-LUAD,2
8,cd14bb97-9a0a-4f06-8ca4-e4105c08a7b5,TCGA-34-5929-01A-01-TS1.222f0d37-3952-46ab-bb8...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-34-5929,TCGA-34-5929-01A,Primary Tumor,Tumor-LUSC,1
9,e5c149b4-85dc-41b2-8251-9386b9c7cf13,TCGA-50-6673-01A-01-BS1.24f3a5ac-0f1d-4977-834...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-50-6673,TCGA-50-6673-01A,Primary Tumor,Tumor-LUAD,2
10,389246a9-4268-42d1-9292-058457133a80,TCGA-53-7626-01A-01-TS1.50562488-8429-4794-970...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-53-7626,TCGA-53-7626-01A,Primary Tumor,Tumor-LUAD,2
...,...,...,...,...,...,...,...,...,...,...
411,dd76a429-9fc2-47db-8ebc-8d6dda0bc99d,TCGA-50-6590-11A-01-TS1.fefe9444-70ce-4911-a03...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-50-6590,TCGA-50-6590-11A,Solid Tissue Normal,Normal,0
412,18f7943d-6a38-4ebc-ab56-354b4980c96a,TCGA-37-4135-01A-01-TS1.fd901f17-fe66-4709-b28...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-37-4135,TCGA-37-4135-01A,Primary Tumor,Tumor-LUSC,1
413,05919b71-18da-47b5-b5fb-af3ff07a871a,TCGA-78-7163-11A-01-TS1.1ce6c71a-1466-48d2-89d...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-78-7163,TCGA-78-7163-11A,Solid Tissue Normal,Normal,0
414,08172153-d520-4d0d-9291-faebfbd4d92d,TCGA-56-8307-01A-01-TS1.7537de75-53be-4ace-aec...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-56-8307,TCGA-56-8307-01A,Primary Tumor,Tumor-LUSC,1


In [62]:
sample = test_df.loc[[0]]
sample

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
0,b05abef8-826a-4381-88a4-9920b2df8d3f,TCGA-60-2725-01A-01-TS1.47dfda47-c072-46cb-80d...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-60-2725,TCGA-60-2725-01A,Primary Tumor,Tumor-LUSC,1


In [63]:
imgage_folder = '/scratch/cz2064/myjupyter/BDML/Project/Data/data_1024/'
file_id = sample['File ID'].values[0]
file_name = sample['File Name'].values[0]
image_path = imgage_folder + file_id + '/' + file_name[:-4] + '_1024.jpg'
image = Image.open(image_path)
image_tensor = preprocess(image).unsqueeze(0)
sample_input = image_tensor.to(device,dtype=torch.float)
sample_lable = sample['label'].values[0]

In [64]:
imgage_folder = '/scratch/cz2064/myjupyter/BDML/Project/Data/data_1024/'
image_path = imgage_folder + file_id + '/' + file_name[:-4] + '_1024.jpg'
original_image = cv2.imread(image_path)

In [65]:
model.eval()
output = model(sample_input)
pre_class = output.data.max(-1)[1].cpu().numpy()

In [66]:
Feature_map = model.featuremap1.cpu().numpy()
print('Feature map:',Feature_map.shape)

Feature map: (1, 512, 32, 32)


In [67]:
params = list(model.parameters())
weight_softmax = np.squeeze(params[-1].data.cpu().numpy())
print('Weights:',weight_softmax.shape)

Weights: (3, 512)


In [68]:
# Feature map 1
weights_map1 = weight_softmax[0]
cam_1 = weights_map1.dot(Feature_map.reshape((512,32*32)))
cam_1 = cam_1.reshape(32,32)
# Feature map 2
weights_map2 = weight_softmax[1]
cam_2 = weights_map2.dot(Feature_map.reshape((512,32*32)))
cam_2 = cam_2.reshape(32,32)
# Feature map 3
weights_map3 = weight_softmax[2]
cam_3 = weights_map3.dot(Feature_map.reshape((512,32*32)))
cam_3 = cam_3.reshape(32,32)

In [69]:
cam = cam_1
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_1.jpg', heatmap*0.3+0.7*original_image)

True

In [70]:
cam = cam_2
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_2.jpg', heatmap*0.3+0.7*original_image)

True

In [71]:
cam = cam_3
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_3.jpg', heatmap*0.3+0.7*original_image)

True

In [72]:
sample_lable

1

In [73]:
pre_class

array([1])